In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir



In [52]:
filepaths = [f for f in listdir("../data") if f.endswith('.xlsx')]


In [53]:
filepaths

['Returnee_MasterList_dataset_DTM_IOMJan 05, 2017.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMAug 31, 2019.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMMay 12, 2016.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMJan 19, 2017.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMOct 31, 2018.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMApr 10, 2015.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMJan 31, 2018.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMFeb 16, 2017.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMMar 02, 2017.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMFeb 29, 2020.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMApr 15, 2018.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMApr 29, 2020.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMFeb 15, 2018.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMNov 05, 2015.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMJul 31, 2018.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMSep 30, 2017.xlsx',
 'Returnee_MasterList_dataset_DTM_IOMMar 30, 2017.xlsx',
 'Returnee_MasterList_dataset_D

In [61]:
df = pd.concat((pd.read_excel("../data/"+f).assign(date=f[35:47]) for f in filepaths))

In [62]:
df

,Location ID,Place_ID,Governorate,District,Location_name,Arabic_name,Latitude,Longitude,Returnee Families,Returnee Individuals,...,Pre June14 Period of displacement,June July14 Period of displacement,August14 Period of displacement,Post September 14 Period of displacement,Post April15 Period of displacement,Post March 16 Period of displacement,Post 17 October 16 Period of displacement,July 17 Period of displacement,Jan19,date
0,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,705.0,4230.0,...,0.0,0.0,705.0,0.0,0.0,0.0,0.0,0.0,0.0,"Jan 05, 2017"
1,1204011.0,13674.0,Erbil,Makhmur,Shahidan,شهيدان,35.780756,43.585956,510.0,3060.0,...,0.0,0.0,510.0,0.0,0.0,0.0,0.0,0.0,0.0,"Jan 05, 2017"
2,1204016.0,13501.0,Erbil,Makhmur,Kurdistan,كوردستان,35.767176,43.576238,280.0,1680.0,...,0.0,0.0,280.0,0.0,0.0,0.0,0.0,0.0,0.0,"Jan 05, 2017"
3,1204017.0,13675.0,Erbil,Makhmur,Sherwany,شيرواني,35.775643,43.584228,660.0,3960.0,...,0.0,0.0,660.0,0.0,0.0,0.0,0.0,0.0,0.0,"Jan 05, 2017"
4,1204018.0,27244.0,Erbil,Makhmur,Saray(markez bazar),سراي(مركز بازار),35.775451,43.577269,218.0,1308.0,...,0.0,0.0,218.0,0.0,0.0,0.0,0.0,0.0,0.0,"Jan 05, 2017"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,2809030.0,27239.0,Salah al-Din,Tuz Khurmatu,Hay Al-Teen,حي التين,34.725080,44.580515,43.0,258.0,...,43.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Aug 17, 2016"
470,2809031.0,27240.0,Salah al-Din,Tuz Khurmatu,Hay Sabaa Tamoz,حي سبعة تموز,34.724910,44.590226,20.0,120.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Aug 17, 2016"
471,2809032.0,27241.0,Salah al-Din,Tuz Khurmatu,Hay Wahid Huzayran,حي واحد حزيران,34.719240,44.581716,30.0,180.0,...,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Aug 17, 2016"
472,2809033.0,27242.0,Salah al-Din,Tuz Khurmatu,Qara Naz village,قرية قرناز,34.785884,44.570116,186.0,1116.0,...,170.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Aug 17, 2016"
